In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
import torch
torch.__version__

'1.7.0'

In [3]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader,TensorDataset,random_split
from torchvision.transforms import transforms

In [4]:
def get_train_labels(train_label):
    for i,num in enumerate(train_label):
        if(num> 4.5):
            train_label[i] = 1
        else:
            train_label[i]= 0
            
    return train_label

In [5]:
with open('../input/stft-research/SFFTFinalData.npy', 'rb') as training_data:
     X= np.load(training_data)

In [6]:
X.shape

(147966, 70)

In [7]:
with open('../input/stft-research/SFFTFinalLabels.npy', 'rb') as training_labels:
     Y= np.load(training_labels)

In [8]:
Y.shape

(147966, 2)

In [9]:
arousal_labels = np.ravel(Y[:, [0]])
valence_labels = np.ravel(Y[:, [1]])

In [10]:
arousal_labels=get_train_labels(arousal_labels)
valence_labels = get_train_labels(valence_labels)

In [46]:
class SFTT(nn.Module):

    def __init__(self):
        super().__init__()
        self.network = nn.Sequential(
            nn.Conv1d(1,64,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm1d(64),
            nn.ReLU(),
            nn.MaxPool1d(2,2),
            nn.Conv1d(64,128,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm1d(128),
            nn.ReLU(),
            nn.MaxPool1d(2,2),

            nn.Conv1d(128,256,kernel_size=3,stride=1,padding=1),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.MaxPool1d(2,2),
            nn.AdaptiveAvgPool1d(1),

            nn.Flatten(),
            nn.Linear(256,128),
            nn.Tanh(),
            nn.Linear(128,64),
            nn.Tanh(),
            nn.Linear(64,1),
            nn.Sigmoid()
    )

    def forward(self,xb):
        out = self.network(xb)
        return out
  
    def training_step(self,batch):
        features,label = batch
        out = self(features)
        loss = F.binary_cross_entropy(out,label)
        return loss

    def validation_step(self,batch):
        features,label = batch
        out = self(features)
        loss = F.binary_cross_entropy(out,label)
        acc = accuracy(out,label)
        return {"val_loss": loss.detach(),"val_acc": acc}

    def validation_epoch_end(self,outputs):
        batch_loss = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_loss).mean()
        batch_acc = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_acc).mean()
        return {"val_loss":epoch_loss.item(),"val_acc":epoch_acc.item()}

    def epoch_end(self,num_epoch,results):
        print("num_epoch: {}, train_loss: {:.2f}, val_loss: {:.2f}, val_acc: {:.2f}".format(num_epoch+1,results['train_loss'],results['val_loss'], results['val_acc']))

In [47]:
def accuracy(out,label):
    out = (out>0.5)
    pred = (out == label).sum()
    return pred/out.shape[0]

def evaluate(model,val_loader):
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

def fit(num_epochs,lr,train_loader,val_loader,model,opt_func=torch.optim.Adam):
    optimizer = opt_func(model.parameters(),lr)
    history = []
    for epoch in range(num_epochs):
        train_losses = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        results = evaluate(model,val_loader)
        train_loss = torch.stack(train_losses).mean().item()
        results['train_loss'] = train_loss
        model.epoch_end(epoch,results)
        history.append(results)
    return history

In [48]:
def get_default_device():
    """Pick GPU if available, else CPU"""
    if torch.cuda.is_available():
        return torch.device('cuda')
    else:
        return torch.device('cpu')
    
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dl, device):
        self.dl = dl
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dl: 
            yield to_device(b, self.device)

    def __len__(self):
        """Number of batches"""
        return len(self.dl)

# Splitting the Dataset and Training

In [49]:
x_a_train,x_a_test,y_a_train,y_a_test=train_test_split(X,np.array(arousal_labels), test_size=0.3,random_state=42)
x_v_train,x_v_test,y_v_train,y_v_test=train_test_split(X,np.array(valence_labels), test_size=0.3,random_state=42)

In [50]:
scaler = StandardScaler()
x_a_train = scaler.fit_transform(x_a_train)
x_a_test = scaler.fit_transform(x_a_test)

x_v_train = scaler.fit_transform(x_v_train)
x_v_test = scaler.fit_transform(x_v_test)

In [51]:
train_a_tensor = torch.tensor(x_a_train,dtype=torch.float32)
train_a_label = torch.tensor(y_a_train,dtype=torch.float32)

test_a_tensor =torch.tensor(x_a_test,dtype=torch.float32)
test_a_label = torch.tensor(y_a_test,dtype=torch.float32)


train_v_tensor = torch.tensor(x_v_train,dtype=torch.float32)
train_v_label = torch.tensor(y_v_train,dtype=torch.float32)

test_v_tensor =torch.tensor(x_v_test,dtype=torch.float32)
test_v_label = torch.tensor(y_v_test,dtype=torch.float32)

In [52]:
train_a_label=train_a_label.unsqueeze(1)
test_a_label=test_a_label.unsqueeze(1)

train_v_label=train_v_label.unsqueeze(1)
test_v_label=test_v_label.unsqueeze(1)

In [53]:
train_a_tensor=train_a_tensor.reshape(train_a_tensor.shape[0],1,70)
test_a_tensor=test_a_tensor.reshape(test_a_tensor.shape[0],1,70)

train_v_tensor=train_v_tensor.reshape(train_v_tensor.shape[0],1,70)
test_v_tensor=test_v_tensor.reshape(test_v_tensor.shape[0],1,70)

In [54]:
print("Train Tensor shape",train_a_tensor.shape)
print("Test Tensor shape",test_a_tensor.shape)

Train Tensor shape torch.Size([103576, 1, 70])
Test Tensor shape torch.Size([44390, 1, 70])


In [55]:
train_arousal = TensorDataset(train_a_tensor, train_a_label)
test_arousal = TensorDataset(test_a_tensor, test_a_label)

train_valence = TensorDataset(train_v_tensor, train_v_label)
test_valence =TensorDataset(test_v_tensor, test_v_label)

In [56]:
device = get_default_device()
device

device(type='cuda')

In [57]:
batch_size = 32
train_arousal_loader =  DataLoader(train_arousal,batch_size=batch_size,shuffle=True)
test_arousal_loader =  DataLoader(test_arousal,batch_size=batch_size,shuffle=True)

batch_size = 32
train_valence_loader =  DataLoader(train_valence,batch_size=batch_size,shuffle=True)
test_valence_loader =  DataLoader(test_valence,batch_size=batch_size,shuffle=True)

In [58]:
train_arousal_loader=DeviceDataLoader(train_arousal_loader,device)
test_arousal_loader=DeviceDataLoader(test_arousal_loader,device)

train_valence_loader = DeviceDataLoader(train_valence_loader,device)
test_valence_loader = DeviceDataLoader(test_valence_loader,device)

In [59]:
arousal_model= SFTT()
arousal_model= arousal_model.to(device)

In [60]:
history = fit(10,0.001,train_arousal_loader,test_arousal_loader,arousal_model)

num_epoch: 1, train_loss: 0.21, val_loss: 0.16, val_acc: 0.93
num_epoch: 2, train_loss: 0.14, val_loss: 0.10, val_acc: 0.95
num_epoch: 3, train_loss: 0.10, val_loss: 0.10, val_acc: 0.96
num_epoch: 4, train_loss: 0.08, val_loss: 0.08, val_acc: 0.97
num_epoch: 5, train_loss: 0.07, val_loss: 0.06, val_acc: 0.98
num_epoch: 6, train_loss: 0.07, val_loss: 0.07, val_acc: 0.97
num_epoch: 7, train_loss: 0.06, val_loss: 0.06, val_acc: 0.97
num_epoch: 8, train_loss: 0.05, val_loss: 0.05, val_acc: 0.98
num_epoch: 9, train_loss: 0.05, val_loss: 0.06, val_acc: 0.97
num_epoch: 10, train_loss: 0.04, val_loss: 0.04, val_acc: 0.98


In [61]:
valence_model= SFTT()
valence_model= valence_model.to(device)

In [62]:
history = fit(10,0.001,train_valence_loader,test_valence_loader,valence_model)

num_epoch: 1, train_loss: 0.34, val_loss: 0.25, val_acc: 0.89
num_epoch: 2, train_loss: 0.22, val_loss: 0.20, val_acc: 0.91
num_epoch: 3, train_loss: 0.17, val_loss: 0.14, val_acc: 0.94
num_epoch: 4, train_loss: 0.14, val_loss: 0.13, val_acc: 0.94
num_epoch: 5, train_loss: 0.13, val_loss: 0.12, val_acc: 0.95
num_epoch: 6, train_loss: 0.11, val_loss: 0.16, val_acc: 0.93
num_epoch: 7, train_loss: 0.10, val_loss: 0.10, val_acc: 0.96
num_epoch: 8, train_loss: 0.09, val_loss: 0.10, val_acc: 0.95
num_epoch: 9, train_loss: 0.08, val_loss: 0.08, val_acc: 0.97
num_epoch: 10, train_loss: 0.08, val_loss: 0.07, val_acc: 0.97
